In [1]:
import wrds

In [18]:
db = wrds.Connection()

Enter your WRDS username [barry]: ksk
Enter your password: ···················


WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [45]:
db.list_libraries()

['wrds_lib_internal',
 'centris',
 'gmi_all_new',
 'secsamp_all_new',
 'compnad',
 'mflinks_all_new',
 'comp_emdb_daily',
 'comp_emdb_monthly',
 'taqm_2018',
 'comp_filings',
 'trown',
 'trsdcgs',
 'twoiq',
 'infogroup_business_academic',
 'crsp_a_treasuries',
 'infogroup_residential',
 'crsp_q_treasuries',
 'risk_governance',
 'crsp_q_stock',
 'crsp_q_ccm',
 'crsp_q_indexhist',
 'risk_proposals',
 'iss_va_shareholder',
 'ff_all_new',
 'crsp_a_indexes',
 'crsp_q_indexes',
 'ciq_keydev',
 'bvd_bvdbankf',
 'lspdsamp',
 'optionmsamp_europe',
 'ktmine_patents_samp',
 'optionmsamp_us',
 'omtrial',
 'ppubsamp_d4d_new',
 'ktsamp',
 'ravenpack_trial',
 'rpnasamp',
 'risk_votes',
 'ciq_ratings',
 'comp_execucomp',
 'taqm_2003',
 'ravenpack_common',
 'taqm_2004',
 'taqm_2008',
 'taqm_2006',
 'taqm_2009',
 'taqm_2010',
 'taqm_2011',
 'taqm_2012',
 'taqm_2013',
 'crsp_q_mutualfunds',
 'bvd_ama_medium',
 'ginsight_basic',
 'ginsight_common',
 'tr_worldscope',
 'levin',
 'trws',
 'comp_snapshot',
 '

In [3]:
db.list_tables(library = 'crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [22]:
import pandas as pd
CCM_link_table = db.get_table(library = 'crsp_q_ccm', table = 'ccmxpf_linktable')
CCM_link_table['linkenddt'] = CCM_link_table['linkenddt'].fillna(pd.datetime.today().date())

In [37]:
CHUNK_SIZE = 1000
OFFSET = 1000
compustat = db.get_table(library = 'compm', table = 'fundq', obs = CHUNK_SIZE, offset=OFFSET)


In [38]:
def merge_permno(row, CCM_link_table):
    gvkey_match = CCM_link_table[CCM_link_table['gvkey'] == row['gvkey']]
    select = gvkey_match[(gvkey_match['linkdt'] < row['datadate']) & (gvkey_match['linkenddt'] > row['datadate'])]
    if len(select) == 0:
        return np.nan
    elif len(select) == 1:
        return select['lpermno'].values[0]
    elif len(select) > 1:
        return '{}  {}  '.format(row['gvkey'], row['datadate'])

In [51]:
merge_permno(row, CCM_link_table)

nan

In [49]:
row = compustat.iloc[90,:]

In [57]:
CCM_link_table[CCM_link_table['gvkey'] == '004162']

,gvkey,linkprim,liid,linktype,lpermno,lpermco,usedflag,linkdt,linkenddt
7638,004162,C,00X,NR,NaN,NaN,-1.0,1982-06-01,1983-10-19
7639,004162,P,01,LC,31318.0,6512.0,1.0,1983-10-20,2002-09-30
7640,004162,J,01,LC,31318.0,6512.0,1.0,2002-10-01,2002-10-21
7641,004162,J,02,LC,78031.0,6512.0,1.0,1992-10-27,2002-10-21
7642,004162,P,03,LC,31318.0,6512.0,1.0,2002-10-22,2008-04-02


In [60]:
list(compustat.columns)

['gvkey',
 'datadate',
 'fyearq',
 'fqtr',
 'fyr',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'tic',
 'cusip',
 'conm',
 'acctchgq',
 'acctstdq',
 'adrrq',
 'ajexq',
 'ajpq',
 'bsprq',
 'compstq',
 'curcdq',
 'curncdq',
 'currtrq',
 'curuscnq',
 'datacqtr',
 'datafqtr',
 'finalq',
 'ogmq',
 'rp',
 'scfq',
 'srcq',
 'staltq',
 'updq',
 'apdedateq',
 'fdateq',
 'pdateq',
 'rdq',
 'acchgq',
 'acomincq',
 'acoq',
 'actq',
 'altoq',
 'ancq',
 'anoq',
 'aociderglq',
 'aociotherq',
 'aocipenq',
 'aocisecglq',
 'aol2q',
 'aoq',
 'apq',
 'aqaq',
 'aqdq',
 'aqepsq',
 'aqpl1q',
 'aqpq',
 'arcedq',
 'arceepsq',
 'arceq',
 'atq',
 'aul3q',
 'billexceq',
 'capr1q',
 'capr2q',
 'capr3q',
 'capsftq',
 'capsq',
 'ceiexbillq',
 'ceqq',
 'cheq',
 'chq',
 'cibegniq',
 'cicurrq',
 'ciderglq',
 'cimiiq',
 'ciotherq',
 'cipenq',
 'ciq',
 'cisecglq',
 'citotalq',
 'cogsq',
 'csh12q',
 'cshfd12',
 'cshfdq',
 'cshiq',
 'cshopq',
 'cshoq',
 'cshprq',
 'cstkcvq',
 'cstkeq',
 'cstkq',
 'dcomq',
 'dd1q',
 'derac